<a href="https://colab.research.google.com/github/rjrizani/BBC/blob/main/BBC_News_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Install dependencies
!pip install -qqq zyte-api==0.7.0 nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.4 MB/s eta 0:00:00


In [2]:

# Imports
from google.colab import userdata
import nest_asyncio
from zyte_api import AsyncZyteAPI

# This is to work with async within a notebook
nest_asyncio.apply()

# Create the Zyte API Client
ZYTE_API_KEY = userdata.get('ZYTE_API_KEY')
print(ZYTE_API_KEY)
client = AsyncZyteAPI(api_key=ZYTE_API_KEY)




082e25fb89434d96b518b211894de5e9


In [25]:

url = "https://www.bbc.com/future/article/20250424-the-body-wrapped-in-plastic-the-image-that-defined-covid-19"

response = await client.get(
    {
        "url": url,
        "browserHtml": True,
        "article": True,
        "articleOptions": {
            "extractFrom": "browserHtml",
        },
        "customAttributes": {
            "articleBody": {
                "type": "string",
                "description": "the body of the article containing the article text. Include as few non-article-body elements as possible, but don't miss any of the article's text",
                "format": "html"  # This is how we specify "extract internally as a reference to the html and don't generate the whole output"
            },
            "fullPageHtml": {
                "type": "string",
                "description": "The full HTML content of the page",
                "format": "html"
            },
            "images_url": {
                "type": "array",
                "description": "the links to all the images in the article",
                "items": {
                    "type": "string",
                    "description": "the links to all the images in the article",
                    "format": "uri"
            }},
            "authorInfo": {
                "type": "string",
                "description": "all the info of the author that wrote the article",
                "format": "html-text"  # We can also indicate to extract only a reference, but to return the text of the html instead of the html
            },
            "relatedArticles": {
                "type": "array",
                "description": "the links to related/other articles in the page",
                "items": {
                    "type": "string",
                    "description": "the links to related/other articles in the page",
                    "format": "uri"  # We can also indicate to extract references to HTML attributes that contain links
                }
            },
            "summary": {
              "type": "string",
              "description": "A two sentence article summary"
            },
            "article_sentiment": {
                "type": "string",
                "enum": ["positive", "negative", "neutral"]
              }

        }
    }
)

In [21]:
response['customAttributes']['metadata']

{'inputTokens': 3434,
 'outputTokens': 273,
 'textInputTokens': 2946,
 'maxInputTokens': 19154,
 'excludedPIIAttributes': []}

In [7]:

print(response['customAttributes']['values']['authorInfo'])

Stephen Dowling


In [8]:
for related_article_url in response['customAttributes']['values']['relatedArticles']:
  print(related_article_url)


https://www.bbc.com/future/article/20250304-the-countries-that-never-locked-down-for-covid-19
https://www.bbc.co.uk/future/article/20250227-covid-19-when-empty-became-the-new-normal
https://www.bbc.co.uk/future/article/20250129-january-2020-the-scanners-used-to-stop-covids-deadly-spread


In [22]:
extracted_html = response['customAttributes']['values']['articleBody']

len(extracted_html)

330

In [26]:
fullPageHtml = response['customAttributes']['values']['fullPageHtml']

len(fullPageHtml)

25152

In [13]:
print(response['customAttributes']['values']['summary'])

An Indonesian photographer captured a defining image of loss due to Covid-19, showing a body wrapped in plastic on a hospital bed. The image was taken in a Jakarta hospital and has been recognized internationally for its powerful representation of the pandemic's toll.


In [17]:
print(response['customAttributes']['values']['images_url'])

['https://ichef.bbci.co.uk/images/ic/480xn/p0l69yw1.jpg.webp', 'https://ichef.bbci.co.uk/images/ic/480xn/p0l69z74.jpg.webp', 'https://ichef.bbci.co.uk/images/ic/480xn/p0l69zg5.jpg.webp']


In [19]:
print(response['customAttributes']['values']['article_sentiment'])

negative


In [27]:
import IPython
IPython.display.HTML(fullPageHtml)